In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import json

In [2]:
#***          Start ib_insync (once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
# ib=IB().connect('127.0.0.1', 7496, clientId=3) # kavi TWS live
ib=IB().connect('127.0.0.1', 4001, clientId=3) # kavi IBG live

In [3]:
#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return np.nan

In [4]:
#******          Symbols Master      *******
#___________________________________________


# NSE symbols from https://www.nseindia.com/products/content/derivatives/equities/fo_underlying_home.htm

index_symbols = 'FTSE100,INDIAVIX,NIFTYCPSE,NIFTY,NIFTYIT,BANKNIFTY,NIFTYMID50,NIFTYPSE,NIFTYINFRA'.split(',')

symbols='ACC,ADANIENT,ADANIPORTS,ADANIPOWER,AJANTPHARM,ALBK,AMARAJABAT,AMBUJACEM,ANDHRABANK,APOLLOHOSP,\
APOLLOTYRE,ARVIND,ASHOKLEY,ASIANPAINT,AUROPHARMA,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,\
BALKRISIND,BALRAMCHIN,BANKBARODA,BANKINDIA,BATAINDIA,BEML,BERGEPAINT,BEL,BHARATFIN,BHARATFORG,\
BPCL,BHARTIARTL,INFRATEL,BHEL,BIOCON,BOSCHLTD,BRITANNIA,CADILAHC,CANFINHOME,CANBK,CAPF,CASTROLIND,\
CEATLTD,CENTURYTEX,CESC,CGPOWER,CHENNPETRO,CHOLAFIN,CIPLA,COALINDIA,COLPAL,CONCOR,CUMMINSIND,DABUR,\
DALMIABHA,DCBBANK,DHFL,DISHTV,DIVISLAB,DLF,DRREDDY,EICHERMOT,ENGINERSIN,EQUITAS,ESCORTS,EXIDEIND,\
FEDERALBNK,GAIL,GLENMARK,GMRINFRA,GODFRYPHLP,GODREJCP,GODREJIND,GRANULES,GRASIM,GSFC,HAVELLS,HCLTECH,\
HDFCBANK,HDFC,HEROMOTOCO,HEXAWARE,HINDALCO,HCC,HINDPETRO,HINDUNILVR,HINDZINC,ICICIBANK,ICICIPRULI,IDBI,\
IDEA,IDFCBANK,IDFC,IFCI,IBULHSGFIN,INDIANB,IOC,IGL,INDUSINDBK,INFIBEAM,INFY,INDIGO,IRB,ITC,JISLJALEQS,\
JPASSOCIAT,JETAIRWAYS,JINDALSTEL,JSWSTEEL,JUBLFOOD,JUSTDIAL,KAJARIACER,KTKBANK,KSCL,KOTAKBANK,KPIT,\
L%26TFH,LT,LICHSGFIN,LUPIN,M%26MFIN,MGL,M%26M,MANAPPURAM,MRPL,MARICO,MARUTI,MFSL,MINDTREE,MOTHERSUMI,\
MRF,MCX,MUTHOOTFIN,NATIONALUM,NBCC,NCC,NESTLEIND,NHPC,NIITTECH,NMDC,NTPC,ONGC,OIL,OFSS,ORIENTBANK,PAGEIND,\
PCJEWELLER,PETRONET,PIDILITIND,PEL,PFC,POWERGRID,PTC,PNB,PVR,RAYMOND,RBLBANK,RELCAPITAL,RCOM,RELIANCE,\
RELINFRA,RPOWER,REPCOHOME,RECLTD,SHREECEM,SRTRANSFIN,SIEMENS,SREINFRA,SRF,SBIN,SAIL,STAR,SUNPHARMA,SUNTV,\
SUZLON,SYNDIBANK,TATACHEM,TATACOMM,TCS,TATAELXSI,TATAGLOBAL,TATAMTRDVR,TATAMOTORS,TATAPOWER,TATASTEEL,TECHM,\
INDIACEM,RAMCOCEM,SOUTHBANK,TITAN,TORNTPHARM,TORNTPOWER,TV18BRDCST,TVSMOTOR,UJJIVAN,ULTRACEMCO,UNIONBANK,\
UBL,MCDOWELL-N,UPL,VEDL,VGUARD,VOLTAS,WIPRO,WOCKPHARMA,YESBANK,ZEEL'.split(',')

In [5]:
#......    !Data Limiter   .........
symbols=symbols[0:10]
#___________________________________

In [6]:
# Rectified for IBKR
isymbols = [i[0:9] for i in symbols]
isymbols = [s.replace('%26','') for s in isymbols]

# Rectified for 5paisa
psymbols = [s.replace('%26', '&') for s in symbols]

In [7]:
%%time
#***     Dictionaries and extracts ***
#_____________________________________

# URLs
urlbase = "https://www.nseindia.com/live_market/dynaContent/live_watch/"
nseurl = urlbase + "get_quote/GetQuote.jsp?symbol="

# Scrape
nsestocks = [nseurl+i for i in symbols]
nserequests = [requests.get(b).text for b in nsestocks]
bs_nses = [BeautifulSoup(s, 'html.parser') for s in nserequests]
nse_data = [f.find(id='responseDiv').text.strip() for f in bs_nses]
nse_dict = [json.loads(bd) for bd in nse_data]

# NSE dictionary extracts
nse_margin = {i['data'][0]['symbol']: float(i['data'][0]['applicableMargin'])/1000 for i in nse_dict}
nse_hi52 = {i['data'][0]['symbol']: float((i['data'][0]['high52']).replace(',','')) for i in nse_dict}
nse_lo52 = {i['data'][0]['symbol']: float((i['data'][0]['low52']).replace(',','')) for i in nse_dict}
nse_lastprice = {i['data'][0]['symbol']: float((i['data'][0]['lastPrice']).replace(',','')) for i in nse_dict}

# MLot and TotalMargin from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol').reset_index(drop=True)
paisa_small = paisa[['Symbol', 'Mlot', 'TotMgnPerLt']].sort_values('Symbol')

paisa_dict = paisa_small.set_index('Symbol').T.to_dict()

Wall time: 33.6 s


In [8]:
%%time
#***   Determine Standard Deviation    ***
#_________________________________________

market = 'NSE'

# qualify contracts
contracts = [Stock(symbol=s, exchange=market) for s in isymbols]
qual_stocks = ib.qualifyContracts(*contracts)

# make a historical dictionary
hist_dict = dict([(c.symbol, ib.reqHistoricalData(contract=c, endDateTime='', durationStr='365 D', 
                                    barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)) for c in contracts])

data = [v for k, v in hist_dict.items()]

sd252 = [catch(lambda: np.std([i.close for i in data[j]][-252:], ddof=1)) 
         for j, k in enumerate(data)] # 252 trading days in a year

sd252_dict = dict(zip(isymbols, sd252))

Wall time: 11.3 s


In [9]:
%%time
#***    Get the RoM  ****
#________________________

# Option chains for qualified stocks
id_symbols = [(s.conId, s.symbol) for s in qual_stocks]

chains = [(symbol, 
           ib.reqSecDefOptParams(futFopExchange='', underlyingConId=cid, underlyingSecType='STK', underlyingSymbol=symbol)) 
          for cid, symbol in id_symbols]

str_exp_dict = dict([(x[0], (sorted(x[1][0].expirations), sorted(x[1][0].strikes))) for x in chains])


# Conversion dictionary for symbols
i2n_symbols = dict(zip(isymbols, psymbols)) #psymbols used because '%26' is converted to '&'...for nse_lastprice

# Filter the strikes
puts_list = [{k: (v[0], i)} 
             for k, v in str_exp_dict.items() 
             for i in v[1] 
             if i < (nse_lastprice[i2n_symbols[k]]- sd252_dict[k]*2)]

calls_list = [{k: (v[0], i)} 
             for k, v in str_exp_dict.items() 
             for i in v[1] 
             if i > (nse_lastprice[i2n_symbols[k]] + sd252_dict[k]*2)]

puts_clist = [{k: ('NSE', e, v[1], 'P')} for puts_dict in puts_list for k, v in puts_dict.items() for e in v[0]]
calls_clist = [{k: ('NSE', e, v[1], 'C')} for calls_dict in calls_list for k, v in calls_dict.items() for e in v[0]]

for puts_dict in puts_clist:
    for k, v in puts_dict.items():
        print(Option(symbol=k, exchange=v[0], lastTradeDateOrContractMonth=v[1], strike=v[2], right=v[3]))

put_option_contracts = [Option(symbol=k, exchange=v[0], lastTradeDateOrContractMonth=v[1], strike=v[2], right=v[3]) 
for puts_dict in puts_clist for k, v in puts_dict.items()]


# Qualify the put contracts
qual_puts = [ib.qualifyContracts(c) for c in put_option_contracts]

qual_puts = [l for i in qual_puts for l in i] # Remove the empty [] entries

live_put_ticks = [t for i in range(0, len(qual_puts), 100) for t in ib.reqTickers(*qual_puts[i:i+100])]

Option(symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=860.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20181025', strike=860.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=860.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=880.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20181025', strike=880.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=880.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=900.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20181025', strike=900.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=900.0, right='P', exchange='NSE')
Option(symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=920.

ERROR:ib_insync.wrapper:Error 200, reqId 34: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181025', strike=860.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181025', strike=860.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 35: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=860.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=860.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 37: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181025', strike=880.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='ACC', lastTradeDateOrContract

ERROR:ib_insync.ib:Unknown contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=1080.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 71: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=1100.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=1100.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 152: No security definition has been found for the request, contract: Option(symbol='ADANIPOWE', lastTradeDateOrContractMonth='20181129', strike=2.5, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='ADANIPOWE', lastTradeDateOrContractMonth='20181129', strike=2.5, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 169: No security definition has been found for the request, contract: Option(symbol='AJANTPHAR', last

ERROR:ib_insync.ib:Unknown contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181025', strike=100.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 266: No security definition has been found for the request, contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181129', strike=100.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181129', strike=100.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 268: No security definition has been found for the request, contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181025', strike=105.0, right='P', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181025', strike=105.0, right='P', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 269: No security definition has been found for the request, contract: Option(symb

Wall time: 4min 4s


In [10]:
live_put_ticks

[Ticker(contract=Option(conId=326004643, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=860.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP860PE', tradingClass='ACC'), time=datetime.datetime(2018, 9, 18, 6, 24, 49, 528441, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=0.0, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[], modelGreeks=OptionComputation(impliedVol=0.4461945947794168, delta=-0.0, optPrice=0.0, pvDividend=0.0, gamma=2.887772174375378e-19, vega=7.994917694259274e-17, theta=-0.0, undPrice=1573.3000000000002)),
 Ticker(contract=Option(conId=326004649, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=880.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP880PE', tradingClass='ACC'), time=datetime.datetime(2018, 9, 18, 6, 24, 49, 528441, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=0.0, ticks=[], tickByTi

In [11]:
bars = [ib.reqHistoricalData(contract=c, endDateTime='', durationStr='1 D', 
                                    barSizeSetting='1 day',  whatToShow='BID_ASK', useRTH=True) for c in qual_puts]

ERROR:ib_insync.wrapper:Error 162, reqId 659: Historical Market Data Service error message:HMDS query returned no data: ACC18SEP860PE@NSE Bid, contract: Option(conId=326004643, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=860.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP860PE', tradingClass='ACC')
ERROR:ib_insync.wrapper:Error 162, reqId 660: Historical Market Data Service error message:HMDS query returned no data: ACC18SEP880PE@NSE Bid, contract: Option(conId=326004649, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=880.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP880PE', tradingClass='ACC')
ERROR:ib_insync.wrapper:Error 162, reqId 661: Historical Market Data Service error message:HMDS query returned no data: ACC18SEP900PE@NSE Bid, contract: Option(conId=324346564, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=900.0, right='P', multiplier='1', exchange='NSE', cur

ERROR:ib_insync.wrapper:Error 162, reqId 688: Historical Market Data Service error message:HMDS query returned no data: ACC18SEP1220PE@NSE Bid, contract: Option(conId=324346695, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=1220.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP1220PE', tradingClass='ACC')
ERROR:ib_insync.wrapper:Error 162, reqId 690: Historical Market Data Service error message:HMDS query returned no data: ACC18NOV1220PE@NSE Bid, contract: Option(conId=331929696, symbol='ACC', lastTradeDateOrContractMonth='20181129', strike=1220.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18NOV1220PE', tradingClass='ACC')
ERROR:ib_insync.wrapper:Error 162, reqId 691: Historical Market Data Service error message:HMDS query returned no data: ACC18SEP1240PE@NSE Bid, contract: Option(conId=324346701, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=1240.0, right='P', multiplier='1', exchange='N

ERROR:ib_insync.wrapper:Error 162, reqId 713: Historical Market Data Service error message:HMDS query returned no data: ADANIPORTS18OCT220PE@NSE Bid, contract: Option(conId=328848988, symbol='ADANIPORT', lastTradeDateOrContractMonth='20181025', strike=220.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ADANIPORTS18OCT220PE', tradingClass='ADANIPORTS')
ERROR:ib_insync.wrapper:Error 162, reqId 714: Historical Market Data Service error message:HMDS query returned no data: ADANIPORTS18NOV220PE@NSE Bid, contract: Option(conId=331907488, symbol='ADANIPORT', lastTradeDateOrContractMonth='20181129', strike=220.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ADANIPORTS18NOV220PE', tradingClass='ADANIPORTS')
ERROR:ib_insync.wrapper:Error 162, reqId 715: Historical Market Data Service error message:HMDS query returned no data: ADANIPORTS18SEP230PE@NSE Bid, contract: Option(conId=324365349, symbol='ADANIPORT', lastTradeDateOrContractMonth='2018

ERROR:ib_insync.wrapper:Error 162, reqId 741: Historical Market Data Service error message:HMDS query returned no data: ADANIPORTS18NOV310PE@NSE Bid, contract: Option(conId=331907563, symbol='ADANIPORT', lastTradeDateOrContractMonth='20181129', strike=310.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ADANIPORTS18NOV310PE', tradingClass='ADANIPORTS')
ERROR:ib_insync.wrapper:Error 162, reqId 744: Historical Market Data Service error message:HMDS query returned no data: ADANIPORTS18NOV320PE@NSE Bid, contract: Option(conId=331907568, symbol='ADANIPORT', lastTradeDateOrContractMonth='20181129', strike=320.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ADANIPORTS18NOV320PE', tradingClass='ADANIPORTS')
ERROR:ib_insync.wrapper:Error 162, reqId 747: Historical Market Data Service error message:HMDS query returned no data: ADANIPORTS18NOV330PE@NSE Bid, contract: Option(conId=331907578, symbol='ADANIPORT', lastTradeDateOrContractMonth='2018

ERROR:ib_insync.wrapper:Error 162, reqId 781: Historical Market Data Service error message:HMDS query returned no data: AJANTPHARM18OCT720PE@NSE Bid, contract: Option(conId=327675588, symbol='AJANTPHAR', lastTradeDateOrContractMonth='20181025', strike=720.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AJANTPHARM18OCT720PE', tradingClass='AJANTPHARM')
ERROR:ib_insync.wrapper:Error 162, reqId 782: Historical Market Data Service error message:HMDS query returned no data: AJANTPHARM18SEP740PE@NSE Bid, contract: Option(conId=325065533, symbol='AJANTPHAR', lastTradeDateOrContractMonth='20180927', strike=740.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AJANTPHARM18SEP740PE', tradingClass='AJANTPHARM')
ERROR:ib_insync.wrapper:Error 162, reqId 783: Historical Market Data Service error message:HMDS query returned no data: AJANTPHARM18OCT740PE@NSE Bid, contract: Option(conId=327675598, symbol='AJANTPHAR', lastTradeDateOrContractMonth='2018

ERROR:ib_insync.wrapper:Error 162, reqId 803: Historical Market Data Service error message:HMDS query returned no data: AMARAJABAT18SEP420PE@NSE Bid, contract: Option(conId=324353206, symbol='AMARAJABA', lastTradeDateOrContractMonth='20180927', strike=420.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AMARAJABAT18SEP420PE', tradingClass='AMARAJABAT')
ERROR:ib_insync.wrapper:Error 162, reqId 804: Historical Market Data Service error message:HMDS query returned no data: AMARAJABAT18SEP440PE@NSE Bid, contract: Option(conId=324353212, symbol='AMARAJABA', lastTradeDateOrContractMonth='20180927', strike=440.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AMARAJABAT18SEP440PE', tradingClass='AMARAJABAT')
ERROR:ib_insync.wrapper:Error 162, reqId 805: Historical Market Data Service error message:HMDS query returned no data: AMARAJABAT18SEP460PE@NSE Bid, contract: Option(conId=324353220, symbol='AMARAJABA', lastTradeDateOrContractMonth='2018

ERROR:ib_insync.wrapper:Error 162, reqId 825: Historical Market Data Service error message:HMDS query returned no data: AMARAJABAT18NOV580PE@NSE Bid, contract: Option(conId=331905250, symbol='AMARAJABA', lastTradeDateOrContractMonth='20181129', strike=580.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AMARAJABAT18NOV580PE', tradingClass='AMARAJABAT')
ERROR:ib_insync.wrapper:Error 162, reqId 826: Historical Market Data Service error message:HMDS query returned no data: AMARAJABAT18SEP600PE@NSE Bid, contract: Option(conId=324353277, symbol='AMARAJABA', lastTradeDateOrContractMonth='20180927', strike=600.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AMARAJABAT18SEP600PE', tradingClass='AMARAJABAT')
ERROR:ib_insync.wrapper:Error 162, reqId 827: Historical Market Data Service error message:HMDS query returned no data: AMARAJABAT18OCT600PE@NSE Bid, contract: Option(conId=327683685, symbol='AMARAJABA', lastTradeDateOrContractMonth='2018

ERROR:ib_insync.wrapper:Error 162, reqId 853: Historical Market Data Service error message:HMDS query returned no data: AMBUJACEM18SEP135PE@NSE Bid, contract: Option(conId=324379087, symbol='AMBUJACEM', lastTradeDateOrContractMonth='20180927', strike=135.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AMBUJACEM18SEP135PE', tradingClass='AMBUJACEM')
ERROR:ib_insync.wrapper:Error 162, reqId 854: Historical Market Data Service error message:HMDS query returned no data: AMBUJACEM18OCT135PE@NSE Bid, contract: Option(conId=327661943, symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181025', strike=135.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='AMBUJACEM18OCT135PE', tradingClass='AMBUJACEM')
ERROR:ib_insync.wrapper:Error 162, reqId 855: Historical Market Data Service error message:HMDS query returned no data: AMBUJACEM18NOV135PE@NSE Bid, contract: Option(conId=332381628, symbol='AMBUJACEM', lastTradeDateOrContractMonth='20181129', 

ERROR:ib_insync.wrapper:Error 162, reqId 881: Historical Market Data Service error message:HMDS query returned no data: APOLLOHOSP18OCT540PE@NSE Bid, contract: Option(conId=327654533, symbol='APOLLOHOS', lastTradeDateOrContractMonth='20181025', strike=540.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='APOLLOHOSP18OCT540PE', tradingClass='APOLLOHOSP')
ERROR:ib_insync.wrapper:Error 162, reqId 882: Historical Market Data Service error message:HMDS query returned no data: APOLLOHOSP18SEP560PE@NSE Bid, contract: Option(conId=326004881, symbol='APOLLOHOS', lastTradeDateOrContractMonth='20180927', strike=560.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='APOLLOHOSP18SEP560PE', tradingClass='APOLLOHOSP')
ERROR:ib_insync.wrapper:Error 162, reqId 883: Historical Market Data Service error message:HMDS query returned no data: APOLLOHOSP18OCT560PE@NSE Bid, contract: Option(conId=327654543, symbol='APOLLOHOS', lastTradeDateOrContractMonth='2018

ERROR:ib_insync.wrapper:Error 162, reqId 903: Historical Market Data Service error message:HMDS query returned no data: APOLLOHOSP18OCT740PE@NSE Bid, contract: Option(conId=327654613, symbol='APOLLOHOS', lastTradeDateOrContractMonth='20181025', strike=740.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='APOLLOHOSP18OCT740PE', tradingClass='APOLLOHOSP')
ERROR:ib_insync.wrapper:Error 162, reqId 904: Historical Market Data Service error message:HMDS query returned no data: APOLLOHOSP18NOV740PE@NSE Bid, contract: Option(conId=332381220, symbol='APOLLOHOS', lastTradeDateOrContractMonth='20181129', strike=740.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='APOLLOHOSP18NOV740PE', tradingClass='APOLLOHOSP')
ERROR:ib_insync.wrapper:Error 162, reqId 905: Historical Market Data Service error message:HMDS query returned no data: APOLLOHOSP18SEP760PE@NSE Bid, contract: Option(conId=324389494, symbol='APOLLOHOS', lastTradeDateOrContractMonth='2018

In [12]:
bars

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [BarData(date=datetime.date(2018, 9, 18), open=1.0, high=1.0, low=1.0, close=1.0, volume=-1, barCount=-1, average=-1.0)],
 [],
 [BarData(date=datetime.date(2018, 9, 18), open=1.0, high=1.0, low=1.0, close=1.0, volume=-1, barCount=-1, average=-1.0)],
 [],
 [],
 [],
 [],
 [],
 [],
 [BarData(date=datetime.date(2018, 9, 18), open=0.15, high=0.15, low=0.15, close=0.15, volume=-1, barCount=-1, average=-1.0)],
 [],
 [],
 [BarData(date=datetime.date(2018, 9, 18), open=0.15, high=0.15, low=0.15, close=0.15, volume=-1, barCount=-1, average=-1.0)],
 [],
 [],
 [BarData(date=datetime.date(2018, 9, 18), open=0.15, high=0.15, low=0.15, close=0.15, volume=-1, barCount=-1, average=-1.0)],
 [],
 [],
 [BarData(date=datetime.date(2018, 9, 18), open=0.15, high=0.15, low=0.15, close=0.15, volume=-1, barCount=-1, average=-1.0)],
 [],
 [],
 [BarData(date=datetime.date(2018, 9, 18), open=0.15, high=0.15, low=0.15, close=0.15, volume=-1, barCount=-1, 

In [13]:
for s, b in zip(qual_puts, bars):
    print(s.symbol, s.lastTradeDateOrContractMonth, s.strike, [(a.open, a.high, a.low, a.close) for a in b])

ACC 20180927 860.0 []
ACC 20180927 880.0 []
ACC 20180927 900.0 []
ACC 20180927 920.0 []
ACC 20180927 940.0 []
ACC 20180927 960.0 []
ACC 20180927 980.0 []
ACC 20180927 1000.0 []
ACC 20180927 1020.0 []
ACC 20180927 1040.0 []
ACC 20180927 1060.0 []
ACC 20180927 1080.0 []
ACC 20180927 1100.0 [(1.0, 1.0, 1.0, 1.0)]
ACC 20181025 1100.0 []
ACC 20180927 1120.0 [(1.0, 1.0, 1.0, 1.0)]
ACC 20181025 1120.0 []
ACC 20181129 1120.0 []
ACC 20180927 1140.0 []
ACC 20181025 1140.0 []
ACC 20181129 1140.0 []
ACC 20180927 1160.0 []
ACC 20181025 1160.0 [(0.15, 0.15, 0.15, 0.15)]
ACC 20181129 1160.0 []
ACC 20180927 1180.0 []
ACC 20181025 1180.0 [(0.15, 0.15, 0.15, 0.15)]
ACC 20181129 1180.0 []
ACC 20180927 1200.0 []
ACC 20181025 1200.0 [(0.15, 0.15, 0.15, 0.15)]
ACC 20181129 1200.0 []
ACC 20180927 1220.0 []
ACC 20181025 1220.0 [(0.15, 0.15, 0.15, 0.15)]
ACC 20181129 1220.0 []
ACC 20180927 1240.0 []
ACC 20181025 1240.0 [(0.15, 0.15, 0.15, 0.15)]
ACC 20181129 1240.0 []
ADANIENT 20180927 90.0 []
ADANIENT 2018102

In [14]:
qual_puts

[Option(conId=326004643, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=860.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP860PE', tradingClass='ACC'),
 Option(conId=326004649, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=880.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP880PE', tradingClass='ACC'),
 Option(conId=324346564, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=900.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP900PE', tradingClass='ACC'),
 Option(conId=324346574, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=920.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP920PE', tradingClass='ACC'),
 Option(conId=324346581, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=940.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP940PE', tradin

In [15]:
for i in qual_puts:
    print(i, paisa_dict[i.symbol]['Mlot'], 'Sell')

Option(conId=326004643, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=860.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP860PE', tradingClass='ACC') 400.0 Sell
Option(conId=326004649, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=880.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP880PE', tradingClass='ACC') 400.0 Sell
Option(conId=324346564, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=900.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP900PE', tradingClass='ACC') 400.0 Sell
Option(conId=324346574, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=920.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18SEP920PE', tradingClass='ACC') 400.0 Sell
Option(conId=324346581, symbol='ACC', lastTradeDateOrContractMonth='20180927', strike=940.0, right='P', multiplier='1', exchange='NSE', currency='INR', 

KeyError: 'ADANIPORT'

# Things to do...
* get option prices
* make qual_calls
* merge with margin to find RoM
* sort on RoM
* place orders
* pickle!

In [ ]:
ib.disconnect()